### Reviewer #1 suggested to compute the 'average DockQ for all 5 models compared with each other'. This notebook intends to compute the DockQ between all the predicted models, i.e. ranked_0 with ranked_1, ranked_0 with ranked_2, ..., until ranked_3 with ranked_4. The same will be done on the minimal RRS.
Author: Chop Yan Lee

In [2]:
import db_utils, os, itertools, subprocess
import numpy as np
import pandas as pd
connect = db_utils.get_connection()

In [35]:
# get prediction names from MySQL DB
query = """select prediction_name, model_id, DockQ from chopyan_db.AlphaFold_prediction_result
where run_id = 'run37' and project_name = 'AlphaFold_benchmark' and model_id = 'ranked_0';"""
ranked_0_df = pd.read_sql_query(query,con=connect)
ranked_0_df

,prediction_name,model_id,DockQ
0,run37_DEG_APCC_KENBOX_2_4GGD,ranked_0,0.878
1,run37_DEG_COP1_1_5IGO,ranked_0,0.203
2,run37_DEG_Kelch_Keap1_1_2FLU,ranked_0,0.957
3,run37_DEG_Kelch_Keap1_2_3WN7,ranked_0,0.628
4,run37_DEG_MDM2_SWIB_1_1YCR,ranked_0,0.928
...,...,...,...
131,run37_TRG_ER_FFAT_1_2RR3,ranked_0,0.832
132,run37_TRG_LysEnd_GGAAcLL_1_1JWG,ranked_0,0.950
133,run37_TRG_NES_CRM1_1_3GB8,ranked_0,0.683
134,run37_TRG_NLS_Bipartite_1_1PJM,ranked_0,0.838


In [62]:
# set up dataframe to store all the computed DockQ
column_type = {'prediction_name':str,'model_id_mobile':str,'model_id_template':str,'DockQ':float}
df = pd.DataFrame(columns=column_type.keys())
df = df.astype(dtype=column_type)

# set up DockQ paths
DockQ_folder_path = f'{os.path.split(os.getcwd())[0]}/DockQ/'
DockQ_path = os.path.join(DockQ_folder_path,'DockQ.py')

# set up run path and model ids
run_path = '/Volumes/imb-luckgr/projects/dmi_predictor/DMI_AF2_PRS/run37/'
models = ['ranked_0','ranked_1','ranked_2','ranked_3','ranked_4']

# compute the DockQ for all combinations of model id
for prediction_name in ranked_0_df['prediction_name'].to_list():
    for mobile, template in itertools.combinations(models,2):
        mobile_path = f'{run_path}{prediction_name}/{mobile}.pdb'
        template_path = f'{run_path}{prediction_name}/{template}.pdb'
        DockQ_process = ['python3', DockQ_path, mobile_path, template_path,'-short']
        try:
            result = subprocess.run(DockQ_process,capture_output=True,text=True,check=True)
            result = result.stdout.split('\n')[-2]
            metrics = []
            values = []
            for i, ele in enumerate(result.split(' ')[:-3]):
                if i % 2 == 0:
                    metrics.append(ele)
                else:
                    values.append(ele)
            DockQ_metrics = {m:v for m, v in zip(metrics,values)}
            DockQ = DockQ_metrics.get('DockQ')
        except: # some template models do not have 'native' contact for DockQ calculation and throw errors
            DockQ = 0
        print(f'{prediction_name}: Mobile: {mobile} Template: {template} DockQ: {DockQ}')
        df.loc[len(df)] = [prediction_name, mobile, template, DockQ]

run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_0 Template: ranked_1 DockQ: 0.976
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_0 Template: ranked_2 DockQ: 0.947
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_0 Template: ranked_3 DockQ: 0.661
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_0 Template: ranked_4 DockQ: 0.326
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_1 Template: ranked_2 DockQ: 0.884
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_1 Template: ranked_3 DockQ: 0.617
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_1 Template: ranked_4 DockQ: 0.289
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_2 Template: ranked_3 DockQ: 0.666
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_2 Template: ranked_4 DockQ: 0.331
run37_DEG_APCC_KENBOX_2_4GGD: Mobile: ranked_3 Template: ranked_4 DockQ: 0.433
run37_DEG_COP1_1_5IGO: Mobile: ranked_0 Template: ranked_1 DockQ: 0.933
run37_DEG_COP1_1_5IGO: Mobile: ranked_0 Template: ranked_2 DockQ: 0.928
run37_DEG_COP1_1_5IGO: Mobile: ranked_0 Template: ranked_3 DockQ: 

run37_DOC_AGCK_PIF_3_1ATP: Mobile: ranked_2 Template: ranked_4 DockQ: 0.457
run37_DOC_AGCK_PIF_3_1ATP: Mobile: ranked_3 Template: ranked_4 DockQ: 0.353
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_0 Template: ranked_1 DockQ: 0.999
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_0 Template: ranked_2 DockQ: 0.999
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_0 Template: ranked_3 DockQ: 0.999
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_0 Template: ranked_4 DockQ: 0.999
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_1 Template: ranked_2 DockQ: 0.999
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_1 Template: ranked_3 DockQ: 0.998
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_1 Template: ranked_4 DockQ: 0.999
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_2 Template: ranked_3 DockQ: 0.998
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_2 Template: ranked_4 DockQ: 0.999
run37_DOC_ANK_TNKS_1_3TWU: Mobile: ranked_3 Template: ranked_4 DockQ: 0.999
run37_DOC_CDC14_PxL_1_6G84: Mobile: ranked_0 Template: ranked_1 DockQ: 0.893
run37_DOC_C

run37_DOC_MAPK_RevD_3_3TEI: Mobile: ranked_1 Template: ranked_3 DockQ: 0.995
run37_DOC_MAPK_RevD_3_3TEI: Mobile: ranked_1 Template: ranked_4 DockQ: 0.972
run37_DOC_MAPK_RevD_3_3TEI: Mobile: ranked_2 Template: ranked_3 DockQ: 0.992
run37_DOC_MAPK_RevD_3_3TEI: Mobile: ranked_2 Template: ranked_4 DockQ: 0.980
run37_DOC_MAPK_RevD_3_3TEI: Mobile: ranked_3 Template: ranked_4 DockQ: 0.970
run37_DOC_MIT_MIM_1_2JQ9: Mobile: ranked_0 Template: ranked_1 DockQ: 0.284
run37_DOC_MIT_MIM_1_2JQ9: Mobile: ranked_0 Template: ranked_2 DockQ: 0.267
run37_DOC_MIT_MIM_1_2JQ9: Mobile: ranked_0 Template: ranked_3 DockQ: 0.239
run37_DOC_MIT_MIM_1_2JQ9: Mobile: ranked_0 Template: ranked_4 DockQ: 0.208
run37_DOC_MIT_MIM_1_2JQ9: Mobile: ranked_1 Template: ranked_2 DockQ: 0.909
run37_DOC_MIT_MIM_1_2JQ9: Mobile: ranked_1 Template: ranked_3 DockQ: 0.730
run37_DOC_MIT_MIM_1_2JQ9: Mobile: ranked_1 Template: ranked_4 DockQ: 0.597
run37_DOC_MIT_MIM_1_2JQ9: Mobile: ranked_2 Template: ranked_3 DockQ: 0.725
run37_DOC_MIT_M

run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_0 Template: ranked_2 DockQ: 0.968
run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_0 Template: ranked_3 DockQ: 0.844
run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_0 Template: ranked_4 DockQ: 0.815
run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_1 Template: ranked_2 DockQ: 0.970
run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_1 Template: ranked_3 DockQ: 0.840
run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_1 Template: ranked_4 DockQ: 0.809
run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_2 Template: ranked_3 DockQ: 0.856
run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_2 Template: ranked_4 DockQ: 0.815
run37_LIG_ActinCP_CPI_1_3AA0: Mobile: ranked_3 Template: ranked_4 DockQ: 0.960
run37_LIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_0 Template: ranked_1 DockQ: 0.949
run37_LIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_0 Template: ranked_2 DockQ: 0.956
run37_LIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_0 Template: ranked_3 DockQ: 0.858
run37_LIG_ActinCP_TwfCPI_2_7DS2: Mobile: ra

run37_LIG_BH_BH3_1_2ROC: Mobile: ranked_2 Template: ranked_3 DockQ: 0.986
run37_LIG_BH_BH3_1_2ROC: Mobile: ranked_2 Template: ranked_4 DockQ: 0.984
run37_LIG_BH_BH3_1_2ROC: Mobile: ranked_3 Template: ranked_4 DockQ: 0.997
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_0 Template: ranked_1 DockQ: 0.985
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_0 Template: ranked_2 DockQ: 0.999
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_0 Template: ranked_3 DockQ: 0.999
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_0 Template: ranked_4 DockQ: 0.986
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_1 Template: ranked_2 DockQ: 0.998
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_1 Template: ranked_3 DockQ: 0.999
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_1 Template: ranked_4 DockQ: 0.999
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_2 Template: ranked_3 DockQ: 0.999
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_2 Template: ranked_4 DockQ: 0.986
run37_LIG_BIR_III_2_1NW9: Mobile: ranked_3 Template: ranked_4 DockQ: 0.986
run37_LIG_BIR_III_3_1JD5: Mo

...

run37_LIG_PALB2_WD40_1_3EU7: Mobile: ranked_0 Template: ranked_4 DockQ: 0.687
run37_LIG_PALB2_WD40_1_3EU7: Mobile: ranked_1 Template: ranked_2 DockQ: 0.340
run37_LIG_PALB2_WD40_1_3EU7: Mobile: ranked_1 Template: ranked_3 DockQ: 0.410
run37_LIG_PALB2_WD40_1_3EU7: Mobile: ranked_1 Template: ranked_4 DockQ: 0.321
run37_LIG_PALB2_WD40_1_3EU7: Mobile: ranked_2 Template: ranked_3 DockQ: 0.495
run37_LIG_PALB2_WD40_1_3EU7: Mobile: ranked_2 Template: ranked_4 DockQ: 0.674
run37_LIG_PALB2_WD40_1_3EU7: Mobile: ranked_3 Template: ranked_4 DockQ: 0.683
run37_LIG_PAM2_1_1JGN: Mobile: ranked_0 Template: ranked_1 DockQ: 0.990
run37_LIG_PAM2_1_1JGN: Mobile: ranked_0 Template: ranked_2 DockQ: 0.991
run37_LIG_PAM2_1_1JGN: Mobile: ranked_0 Template: ranked_3 DockQ: 0.975
run37_LIG_PAM2_1_1JGN: Mobile: ranked_0 Template: ranked_4 DockQ: 0.943
run37_LIG_PAM2_1_1JGN: Mobile: ranked_1 Template: ranked_2 DockQ: 0.990
run37_LIG_PAM2_1_1JGN: Mobile: ranked_1 Template: ranked_3 DockQ: 0.986
run37_LIG_PAM2_1_1JGN:

run37_LIG_Pex14_1_2W84: Mobile: ranked_0 Template: ranked_1 DockQ: 0.938
run37_LIG_Pex14_1_2W84: Mobile: ranked_0 Template: ranked_2 DockQ: 0.965
run37_LIG_Pex14_1_2W84: Mobile: ranked_0 Template: ranked_3 DockQ: 0.954
run37_LIG_Pex14_1_2W84: Mobile: ranked_0 Template: ranked_4 DockQ: 0.939
run37_LIG_Pex14_1_2W84: Mobile: ranked_1 Template: ranked_2 DockQ: 0.940
run37_LIG_Pex14_1_2W84: Mobile: ranked_1 Template: ranked_3 DockQ: 0.919
run37_LIG_Pex14_1_2W84: Mobile: ranked_1 Template: ranked_4 DockQ: 0.926
run37_LIG_Pex14_1_2W84: Mobile: ranked_2 Template: ranked_3 DockQ: 0.942
run37_LIG_Pex14_1_2W84: Mobile: ranked_2 Template: ranked_4 DockQ: 0.984
run37_LIG_Pex14_1_2W84: Mobile: ranked_3 Template: ranked_4 DockQ: 0.937
run37_LIG_Pex14_3_4BXU: Mobile: ranked_0 Template: ranked_1 DockQ: 0.740
run37_LIG_Pex14_3_4BXU: Mobile: ranked_0 Template: ranked_2 DockQ: 0.463
run37_LIG_Pex14_3_4BXU: Mobile: ranked_0 Template: ranked_3 DockQ: 0.586
run37_LIG_Pex14_3_4BXU: Mobile: ranked_0 Template: 

run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_0 Template: ranked_1 DockQ: 0.998
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_0 Template: ranked_2 DockQ: 0.980
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_0 Template: ranked_3 DockQ: 0.439
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_0 Template: ranked_4 DockQ: 0.408
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_1 Template: ranked_2 DockQ: 0.983
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_1 Template: ranked_3 DockQ: 0.437
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_1 Template: ranked_4 DockQ: 0.407
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_2 Template: ranked_3 DockQ: 0.443
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_2 Template: ranked_4 DockQ: 0.412
run37_LIG_SH3_CIN85_PxpxPR_1_2BZ8: Mobile: ranked_3 Template: ranked_4 DockQ: 0.959
run37_LIG_Sin3_1_1PD7: Mobile: ranked_0 Template: ranked_1 DockQ: 0.977
run37_LIG_Sin3_1_1PD7: Mobile: ranked_0 Template: ranked_2 DockQ: 0.972
run37_LIG_Sin3_1

run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_0 Template: ranked_1 DockQ: 0.975
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_0 Template: ranked_2 DockQ: 0.986
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_0 Template: ranked_3 DockQ: 0.977
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_0 Template: ranked_4 DockQ: 0.999
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_1 Template: ranked_2 DockQ: 0.986
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_1 Template: ranked_3 DockQ: 0.966
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_1 Template: ranked_4 DockQ: 0.985
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_2 Template: ranked_3 DockQ: 0.977
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_2 Template: ranked_4 DockQ: 0.996
run37_LIG_ULM_U2AF65_1_1O0P: Mobile: ranked_3 Template: ranked_4 DockQ: 0.998
run37_LIG_Vh1_VBS_1_1SYQ: Mobile: ranked_0 Template: ranked_1 DockQ: 0.993
run37_LIG_Vh1_VBS_1_1SYQ: Mobile: ranked_0 Template: ranked_2 DockQ: 0.986
run37_LIG_Vh1_VBS_1_1SYQ: Mobile: ranked_0 Template: ranked_3 DockQ: 0

run37_TRG_DiLeu_BaLyEn_6_2JKR: Mobile: ranked_1 Template: ranked_4 DockQ: 0.699
run37_TRG_DiLeu_BaLyEn_6_2JKR: Mobile: ranked_2 Template: ranked_3 DockQ: 0.459
run37_TRG_DiLeu_BaLyEn_6_2JKR: Mobile: ranked_2 Template: ranked_4 DockQ: 0.649
run37_TRG_DiLeu_BaLyEn_6_2JKR: Mobile: ranked_3 Template: ranked_4 DockQ: 0.519
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_0 Template: ranked_1 DockQ: 0.980
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_0 Template: ranked_2 DockQ: 0.954
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_0 Template: ranked_3 DockQ: 0.967
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_0 Template: ranked_4 DockQ: 0.968
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_1 Template: ranked_2 DockQ: 0.955
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_1 Template: ranked_3 DockQ: 0.953
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_1 Template: ranked_4 DockQ: 0.968
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_2 Template: ranked_3 DockQ: 0.983
run37_TRG_ENDOCYTIC_2_1BXX: Mobile: ranked_2 Template: ranked_4 

In [64]:
df.to_csv('/Users/chopyanlee/Coding/Python/DMI/AlphaFold/DockQ_between_models_run37.tsv',sep='\t')

### Do the same for the minimal RRS run

In [3]:
# get prediction names from MySQL DB
query = """select prediction_name, model_id, DockQ from chopyan_db.AlphaFold_prediction_result
where run_id = 'run38' and project_name = 'AlphaFold_benchmark' and model_id = 'ranked_0';"""
ranked_0_df = pd.read_sql_query(query,con=connect)
ranked_0_df

,prediction_name,model_id,DockQ
0,run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CAR...,ranked_0,None
1,run38_MDEG_COP1_1_5IGO.DDOC_USP7_MATH_1_3MQS,ranked_0,None
2,run38_MDEG_Kelch_Keap1_1_2FLU.DLIG_DLG_GKlike_...,ranked_0,None
3,run38_MDEG_Kelch_Keap1_2_3WN7.DDOC_USP7_MATH_2...,ranked_0,None
4,run38_MDEG_MDM2_SWIB_1_1YCR.DLIG_PCNA_APIM_2_5MLW,ranked_0,None
...,...,...,...
131,run38_MTRG_ER_FFAT_1_2RR3.DDOC_MAPK_HePTP_8_2GPH,ranked_0,None
132,run38_MTRG_LysEnd_GGAAcLL_1_1JWG.DLIG_SUMO_SIM...,ranked_0,None
133,run38_MTRG_NES_CRM1_1_3GB8.DDOC_PP1_MyPhoNE_1_...,ranked_0,None
134,run38_MTRG_NLS_Bipartite_1_1PJM.DLIG_PDZ_Class...,ranked_0,None


In [4]:
# set up dataframe to store all the computed DockQ
column_type = {'prediction_name':str,'model_id_mobile':str,'model_id_template':str,'DockQ':float}
df = pd.DataFrame(columns=column_type.keys())
df = df.astype(dtype=column_type)

# set up DockQ paths
DockQ_folder_path = f'{os.path.split(os.getcwd())[0]}/DockQ/'
DockQ_path = os.path.join(DockQ_folder_path,'DockQ.py')

# set up run path and model ids
run_path = '/Volumes/imb-luckgr/projects/dmi_predictor/DMI_AF2_PRS/run38/'
models = ['ranked_0','ranked_1','ranked_2','ranked_3','ranked_4']

# compute the DockQ for all combinations of model id
for prediction_name in ranked_0_df['prediction_name'].to_list():
    for mobile, template in itertools.combinations(models,2):
        mobile_path = f'{run_path}{prediction_name}/{mobile}.pdb'
        template_path = f'{run_path}{prediction_name}/{template}.pdb'
        DockQ_process = ['python3', DockQ_path, mobile_path, template_path,'-short']
        try:
            result = subprocess.run(DockQ_process,capture_output=True,text=True,check=True)
            result = result.stdout.split('\n')[-2]
            metrics = []
            values = []
            for i, ele in enumerate(result.split(' ')[:-3]):
                if i % 2 == 0:
                    metrics.append(ele)
                else:
                    values.append(ele)
            DockQ_metrics = {m:v for m, v in zip(metrics,values)}
            DockQ = DockQ_metrics.get('DockQ')
        except: # some template models do not have 'native' contact for DockQ calculation and throw errors
            DockQ = 0
        print(f'{prediction_name}: Mobile: {mobile} Template: {template} DockQ: {DockQ}')
        df.loc[len(df)] = [prediction_name, mobile, template, DockQ]

run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_0 Template: ranked_1 DockQ: 0.989
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_0 Template: ranked_2 DockQ: 0.977
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_0 Template: ranked_3 DockQ: 0.917
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_0 Template: ranked_4 DockQ: 0.908
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_1 Template: ranked_2 DockQ: 0.965
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_1 Template: ranked_3 DockQ: 0.921
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_1 Template: ranked_4 DockQ: 0.939
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_2 Template: ranked_3 DockQ: 0.939
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30: Mobile: ranked_2 Template: ranked_4 DockQ: 0.931
run38_MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CAR

run38_MDEG_SIAH_1_2A25.DLIG_CAP-Gly_2_3RDV: Mobile: ranked_0 Template: ranked_4 DockQ: 0.195
run38_MDEG_SIAH_1_2A25.DLIG_CAP-Gly_2_3RDV: Mobile: ranked_1 Template: ranked_2 DockQ: 0.923
run38_MDEG_SIAH_1_2A25.DLIG_CAP-Gly_2_3RDV: Mobile: ranked_1 Template: ranked_3 DockQ: 0.792
run38_MDEG_SIAH_1_2A25.DLIG_CAP-Gly_2_3RDV: Mobile: ranked_1 Template: ranked_4 DockQ: 0.236
run38_MDEG_SIAH_1_2A25.DLIG_CAP-Gly_2_3RDV: Mobile: ranked_2 Template: ranked_3 DockQ: 0.818
run38_MDEG_SIAH_1_2A25.DLIG_CAP-Gly_2_3RDV: Mobile: ranked_2 Template: ranked_4 DockQ: 0.236
run38_MDEG_SIAH_1_2A25.DLIG_CAP-Gly_2_3RDV: Mobile: ranked_3 Template: ranked_4 DockQ: 0.223
run38_MDEG_SPOP_SBC_1_3HQM.DDOC_MIT_MIM_1_2JQ9: Mobile: ranked_0 Template: ranked_1 DockQ: 0.203
run38_MDEG_SPOP_SBC_1_3HQM.DDOC_MIT_MIM_1_2JQ9: Mobile: ranked_0 Template: ranked_2 DockQ: 0.509
run38_MDEG_SPOP_SBC_1_3HQM.DDOC_MIT_MIM_1_2JQ9: Mobile: ranked_0 Template: ranked_3 DockQ: 0.169
run38_MDEG_SPOP_SBC_1_3HQM.DDOC_MIT_MIM_1_2JQ9: Mobile: ra

run38_MDOC_MAPK_GRA24_9_5ETA.DLIG_PCNA_APIM_2_5MLW: Mobile: ranked_2 Template: ranked_3 DockQ: 0.175
run38_MDOC_MAPK_GRA24_9_5ETA.DLIG_PCNA_APIM_2_5MLW: Mobile: ranked_2 Template: ranked_4 DockQ: 0.173
run38_MDOC_MAPK_GRA24_9_5ETA.DLIG_PCNA_APIM_2_5MLW: Mobile: ranked_3 Template: ranked_4 DockQ: 0.658
run38_MDOC_MAPK_HePTP_8_2GPH.DLIG_PCNA_yPIPBox_3_1SXJ: Mobile: ranked_0 Template: ranked_1 DockQ: 0.747
run38_MDOC_MAPK_HePTP_8_2GPH.DLIG_PCNA_yPIPBox_3_1SXJ: Mobile: ranked_0 Template: ranked_2 DockQ: 0.764
run38_MDOC_MAPK_HePTP_8_2GPH.DLIG_PCNA_yPIPBox_3_1SXJ: Mobile: ranked_0 Template: ranked_3 DockQ: 0.604
run38_MDOC_MAPK_HePTP_8_2GPH.DLIG_PCNA_yPIPBox_3_1SXJ: Mobile: ranked_0 Template: ranked_4 DockQ: 0.576
run38_MDOC_MAPK_HePTP_8_2GPH.DLIG_PCNA_yPIPBox_3_1SXJ: Mobile: ranked_1 Template: ranked_2 DockQ: 0.702
run38_MDOC_MAPK_HePTP_8_2GPH.DLIG_PCNA_yPIPBox_3_1SXJ: Mobile: ranked_1 Template: ranked_3 DockQ: 0.513
run38_MDOC_MAPK_HePTP_8_2GPH.DLIG_PCNA_yPIPBox_3_1SXJ: Mobile: ranked_1 T

run38_MDOC_PP1_RVXF_1_3N5U.DLIG_14-3-3_ChREBP_3_5F74: Mobile: ranked_3 Template: ranked_4 DockQ: 0.978
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_0 Template: ranked_1 DockQ: 0.359
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_0 Template: ranked_2 DockQ: 0.266
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_0 Template: ranked_3 DockQ: 0.017
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_0 Template: ranked_4 DockQ: 0.017
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_1 Template: ranked_2 DockQ: 0.497
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_1 Template: ranked_3 DockQ: 0.017
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_1 Template: ranked_4 DockQ: 0.018
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_2 Template: ranked_3 DockQ: 0.014
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_2 Template: ranked_4 DockQ: 0.014
run38_MDOC_PP1_SILK_1_2O8G.DLIG_WW_1_1EG4: Mobile: ranked_3 Template:

run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_0 Template: ranked_2 DockQ: 0.863
run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_0 Template: ranked_3 DockQ: 0.659
run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_0 Template: ranked_4 DockQ: 0.886
run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_1 Template: ranked_2 DockQ: 0.801
run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_1 Template: ranked_3 DockQ: 0.631
run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_1 Template: ranked_4 DockQ: 0.915
run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_2 Template: ranked_3 DockQ: 0.606
run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_2 Template: ranked_4 DockQ: 0.807
run38_MLIG_Actin_RPEL_3_2V52.DLIG_ActinCP_TwfCPI_2_7DS2: Mobile: ranked_3 Template: ranked_4 DockQ: 0.626
run38_MLIG_Actin_WH2_1_2A41.DLIG_GBD_Chelix_1_

...

run38_MLIG_PROFILIN_1_2V8C.DDOC_PP2B_PxIxI_1_4F0Z: Mobile: ranked_1 Template: ranked_2 DockQ: 0.224
run38_MLIG_PROFILIN_1_2V8C.DDOC_PP2B_PxIxI_1_4F0Z: Mobile: ranked_1 Template: ranked_3 DockQ: 0.377
run38_MLIG_PROFILIN_1_2V8C.DDOC_PP2B_PxIxI_1_4F0Z: Mobile: ranked_1 Template: ranked_4 DockQ: 0.227
run38_MLIG_PROFILIN_1_2V8C.DDOC_PP2B_PxIxI_1_4F0Z: Mobile: ranked_2 Template: ranked_3 DockQ: 0.273
run38_MLIG_PROFILIN_1_2V8C.DDOC_PP2B_PxIxI_1_4F0Z: Mobile: ranked_2 Template: ranked_4 DockQ: 0.823
run38_MLIG_PROFILIN_1_2V8C.DDOC_PP2B_PxIxI_1_4F0Z: Mobile: ranked_3 Template: ranked_4 DockQ: 0.284
run38_MLIG_PTAP_UEV_1_1M4P.DLIG_LSD1_SNAG_1_2Y48: Mobile: ranked_0 Template: ranked_1 DockQ: 0.832
run38_MLIG_PTAP_UEV_1_1M4P.DLIG_LSD1_SNAG_1_2Y48: Mobile: ranked_0 Template: ranked_2 DockQ: 0.798
run38_MLIG_PTAP_UEV_1_1M4P.DLIG_LSD1_SNAG_1_2Y48: Mobile: ranked_0 Template: ranked_3 DockQ: 0.591
run38_MLIG_PTAP_UEV_1_1M4P.DLIG_LSD1_SNAG_1_2Y48: Mobile: ranked_0 Template: ranked_4 DockQ: 0.662
run3

run38_MLIG_SH3_3_2GBQ.DLIG_IBAR_NPY_1_2YKT: Mobile: ranked_2 Template: ranked_3 DockQ: 0.176
run38_MLIG_SH3_3_2GBQ.DLIG_IBAR_NPY_1_2YKT: Mobile: ranked_2 Template: ranked_4 DockQ: 0.452
run38_MLIG_SH3_3_2GBQ.DLIG_IBAR_NPY_1_2YKT: Mobile: ranked_3 Template: ranked_4 DockQ: 0.138
run38_MLIG_SH3_CIN85_PxpxPR_1_2BZ8.DLIG_PAM2_1_1JGN: Mobile: ranked_0 Template: ranked_1 DockQ: 0.070
run38_MLIG_SH3_CIN85_PxpxPR_1_2BZ8.DLIG_PAM2_1_1JGN: Mobile: ranked_0 Template: ranked_2 DockQ: 0.295
run38_MLIG_SH3_CIN85_PxpxPR_1_2BZ8.DLIG_PAM2_1_1JGN: Mobile: ranked_0 Template: ranked_3 DockQ: 0.060
run38_MLIG_SH3_CIN85_PxpxPR_1_2BZ8.DLIG_PAM2_1_1JGN: Mobile: ranked_0 Template: ranked_4 DockQ: 0.077
run38_MLIG_SH3_CIN85_PxpxPR_1_2BZ8.DLIG_PAM2_1_1JGN: Mobile: ranked_1 Template: ranked_2 DockQ: 0.035
run38_MLIG_SH3_CIN85_PxpxPR_1_2BZ8.DLIG_PAM2_1_1JGN: Mobile: ranked_1 Template: ranked_3 DockQ: 0.232
run38_MLIG_SH3_CIN85_PxpxPR_1_2BZ8.DLIG_PAM2_1_1JGN: Mobile: ranked_1 Template: ranked_4 DockQ: 0.126
run38_M

run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_0 Template: ranked_2 DockQ: 0.963
run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_0 Template: ranked_3 DockQ: 0.951
run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_0 Template: ranked_4 DockQ: 0.870
run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_1 Template: ranked_2 DockQ: 0.976
run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_1 Template: ranked_3 DockQ: 0.968
run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_1 Template: ranked_4 DockQ: 0.918
run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_2 Template: ranked_3 DockQ: 0.904
run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_2 Template: ranked_4 DockQ: 0.852
run38_MLIG_TRFH_1_3BQO.DLIG_UFM1_UFIM_1_5HKH: Mobile: ranked_3 Template: ranked_4 DockQ: 0.908
run38_MLIG_UFM1_UFIM_1_5HKH.DLIG_KLC1_Yacidic_2_6FUZ: Mobile: ranked_0 Template: ranked_1 DockQ: 0.967
run38_MLIG_UFM1_UFIM_1_5HKH.DLIG_KLC1_Yaci

run38_MMOD_PKB_1_1O6K.DLIG_PDZ_Class_2_1MFG: Mobile: ranked_0 Template: ranked_4 DockQ: 0.813
run38_MMOD_PKB_1_1O6K.DLIG_PDZ_Class_2_1MFG: Mobile: ranked_1 Template: ranked_2 DockQ: 0.609
run38_MMOD_PKB_1_1O6K.DLIG_PDZ_Class_2_1MFG: Mobile: ranked_1 Template: ranked_3 DockQ: 0.680
run38_MMOD_PKB_1_1O6K.DLIG_PDZ_Class_2_1MFG: Mobile: ranked_1 Template: ranked_4 DockQ: 0.611
run38_MMOD_PKB_1_1O6K.DLIG_PDZ_Class_2_1MFG: Mobile: ranked_2 Template: ranked_3 DockQ: 0.802
run38_MMOD_PKB_1_1O6K.DLIG_PDZ_Class_2_1MFG: Mobile: ranked_2 Template: ranked_4 DockQ: 0.780
run38_MMOD_PKB_1_1O6K.DLIG_PDZ_Class_2_1MFG: Mobile: ranked_3 Template: ranked_4 DockQ: 0.791
run38_MMOD_SUMO_for_1_1KPS.DDOC_MAPK_DCC_7_2B9J: Mobile: ranked_0 Template: ranked_1 DockQ: 0.958
run38_MMOD_SUMO_for_1_1KPS.DDOC_MAPK_DCC_7_2B9J: Mobile: ranked_0 Template: ranked_2 DockQ: 0.609
run38_MMOD_SUMO_for_1_1KPS.DDOC_MAPK_DCC_7_2B9J: Mobile: ranked_0 Template: ranked_3 DockQ: 0.509
run38_MMOD_SUMO_for_1_1KPS.DDOC_MAPK_DCC_7_2B9J:

run38_MTRG_NES_CRM1_1_3GB8.DDOC_PP1_MyPhoNE_1_1S70: Mobile: ranked_1 Template: ranked_2 DockQ: 0.572
run38_MTRG_NES_CRM1_1_3GB8.DDOC_PP1_MyPhoNE_1_1S70: Mobile: ranked_1 Template: ranked_3 DockQ: 0.143
run38_MTRG_NES_CRM1_1_3GB8.DDOC_PP1_MyPhoNE_1_1S70: Mobile: ranked_1 Template: ranked_4 DockQ: 0.204
run38_MTRG_NES_CRM1_1_3GB8.DDOC_PP1_MyPhoNE_1_1S70: Mobile: ranked_2 Template: ranked_3 DockQ: 0.124
run38_MTRG_NES_CRM1_1_3GB8.DDOC_PP1_MyPhoNE_1_1S70: Mobile: ranked_2 Template: ranked_4 DockQ: 0.186
run38_MTRG_NES_CRM1_1_3GB8.DDOC_PP1_MyPhoNE_1_1S70: Mobile: ranked_3 Template: ranked_4 DockQ: 0.497
run38_MTRG_NLS_Bipartite_1_1PJM.DLIG_PDZ_Class_1_1D5G: Mobile: ranked_0 Template: ranked_1 DockQ: 0.673
run38_MTRG_NLS_Bipartite_1_1PJM.DLIG_PDZ_Class_1_1D5G: Mobile: ranked_0 Template: ranked_2 DockQ: 0.704
run38_MTRG_NLS_Bipartite_1_1PJM.DLIG_PDZ_Class_1_1D5G: Mobile: ranked_0 Template: ranked_3 DockQ: 0.071
run38_MTRG_NLS_Bipartite_1_1PJM.DLIG_PDZ_Class_1_1D5G: Mobile: ranked_0 Template: 

In [5]:
df.to_csv('/Users/chopyanlee/AlphaFold_benchmark/DockQ_between_models_run38.tsv',sep='\t')